In [532]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import preprocessing,model_selection
from sklearn.model_selection import train_test_split,StratifiedKFold
from sklearn.metrics import accuracy_score, mean_squared_error, mean_squared_log_error
from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.svm import SVR
pd.set_option('display.max_columns', 500)

In [533]:
train = pd.read_excel('Data_Train.xlsx')
test = pd.read_excel('Data_Test.xlsx')

In [534]:
print (train.shape, test.shape)

(6019, 13) (1234, 12)


In [535]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6019 entries, 0 to 6018
Data columns (total 13 columns):
Name                 6019 non-null object
Location             6019 non-null object
Year                 6019 non-null int64
Kilometers_Driven    6019 non-null int64
Fuel_Type            6019 non-null object
Transmission         6019 non-null object
Owner_Type           6019 non-null object
Mileage              6017 non-null object
Engine               5983 non-null object
Power                5983 non-null object
Seats                5977 non-null float64
New_Price            824 non-null object
Price                6019 non-null float64
dtypes: float64(2), int64(2), object(9)
memory usage: 611.4+ KB


In [536]:
train.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,Manual,First,26.6 km/kg,998 CC,58.16 bhp,5.0,NaN,1.75
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,NaN,12.50
2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.2 kmpl,1199 CC,88.7 bhp,5.0,8.61 Lakh,4.50
3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,NaN,6.00
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,NaN,17.74


In [537]:
def data_cleaning(df):
    df['Name'] = df['Name'].apply(lambda x: x.lower())
    df['Name'] = df['Name'].apply(lambda x: x.replace('land rover','landrover'))
    df['Name'] = df['Name'].apply(lambda x: x.replace('s cross ','s-cross '))
    df['Name'] = df['Name'].apply(lambda x: x.replace('mahindra e ','mahindra verito '))
    df['Name'] = df['Name'].apply(lambda x: x.replace('mercedes-benz new ','mercedes-benz c-class '))
    df['Name'] = df['Name'].apply(lambda x: x.replace('mini countryman ','mini cooper '))
    df['Name'] = df['Name'].apply(lambda x: x.replace('mini clubman ','mini cooper '))
    df['Name'] = df['Name'].apply(lambda x: x.replace('wr-v','wrv'))
    df['Name'] = df['Name'].apply(lambda x: x.replace('br-v','brv'))
    
    df['Manufacturer'] = df['Name'].apply(lambda x:x.split(" ")[0].lower())
    df['Model'] = df['Name'].apply(lambda x:x.split(" ")[1].lower())
    df["Manu_Model"] = df["Manufacturer"].astype(str) + "_" + df["Model"].astype('str')
    
    df['old_by'] = df['Year'].apply(lambda x: 2019 - int(x)) 

    df['Mileage'] = df['Mileage'].apply(lambda x: float(str(x).split(" ")[0]))
    df['Mileage'] = np.where(df['Fuel_Type'] == 'Diesel',(df['Mileage']/0.832),df['Mileage'])
    df['Mileage'] = np.where(df['Fuel_Type'] == 'Petrol',(df['Mileage']/0.740),df['Mileage'])
    df['Mileage'].replace(0.0, np.mean(df['Mileage']), inplace=True)

    df['Engine'].fillna('0 cc', inplace=True)
    df['Engine'] = df['Engine'].apply(lambda x : int(str(x).split(" ")[0]))
    df['Engine'].replace(0, np.median(df['Engine']), inplace=True)

    df['Power'] = df['Power'].replace('null bhp','0.0 bhp')
    df['Power'].fillna('0.0 bhp', inplace=True)
    df['Power'] = df['Power'].apply(lambda x : float(str(x).split(" ")[0]))
    df['Power'].replace(0.0, np.median(df['Power']), inplace=True)

    df['Seats'].fillna(0.0, inplace=True)
    df['Seats'] = df['Seats'].replace(0.0, 5.0)
    
    return df

In [538]:
train = data_cleaning(train)
test = data_cleaning(test)

In [539]:
train.loc[train['New_Price'].isnull(),'New_Price'] = '0.0 Lakh'
train['New_Price'] = train['New_Price'].apply(lambda x : float(x.split(" ")[0]) if str(x.split(" ")[1]) == 'Lakh' else float(x.split(" ")[0])*100) 

test.loc[test['New_Price'].isnull(),'New_Price'] = '0.0 Lakh'
test['New_Price'] = test['New_Price'].apply(lambda x : float(x.split(" ")[0]) if str(x.split(" ")[1]) == 'Lakh' else float(x.split(" ")[0])*100) 

In [540]:
train_copy = train.drop(['Price'],axis=1)
df = train_copy.append(test)
print (df.shape)

(7253, 16)


In [541]:
temp_zero = df[df['New_Price'] != 0.0]
temp_missing = df[df['New_Price'] == 0.0]

In [542]:
for col in ['Manufacturer','Model','Manu_Model']:
    mdf = temp_zero.groupby([col, "Seats"])['New_Price'].mean().reset_index()
    mdf.columns = [col,"Seats",col+"_Seats_mean_Price"]
    train = pd.merge(train, mdf, on=[col,"Seats"], how="left")
    test = pd.merge(test, mdf, on=[col,"Seats"], how="left")
    
    mdf = temp_zero.groupby([col,'Seats'])['New_Price'].std().reset_index()
    mdf.columns = [col,"Seats",col+"_Seats_std_Price"]
    train = pd.merge(train, mdf, on=[col,"Seats"], how="left")
    test = pd.merge(test, mdf, on=[col,"Seats"], how="left")

In [543]:
mdf = temp_zero.groupby(["Model", "Year"])['New_Price'].mean().reset_index()
mdf.columns = ["Model",'Year',"Model_Year_mean_Price"]
train = pd.merge(train, mdf, on=["Model",'Year'], how="left")
test = pd.merge(test, mdf, on=["Model",'Year'], how="left")

train["Model_Year_mean_Price"].fillna(0, inplace=True)
test["Model_Year_mean_Price"].fillna(0, inplace=True)

In [544]:
for col in ['Manufacturer','Model']:
    mdf = temp_zero.groupby(col)['New_Price'].mean().reset_index()
    mdf.columns = [col,col+"_mean_Price"]
    train = pd.merge(train, mdf, on=[col], how="left")
    test = pd.merge(test, mdf, on=[col], how="left")

    mdf = temp_zero.groupby([col])['New_Price'].std().reset_index()
    mdf.columns = [col,col+"_std_Price"]
    train = pd.merge(train, mdf, on=[col], how="left")
    test = pd.merge(test, mdf, on=[col], how="left")


In [545]:
collist = ['Manufacturer_Seats_mean_Price', 'Manufacturer_Seats_std_Price',
           'Model_Seats_mean_Price', 'Model_Seats_std_Price',
           'Manu_Model_Seats_mean_Price', 'Manu_Model_Seats_std_Price',
           'Manufacturer_mean_Price', 'Manufacturer_std_Price', 'Model_mean_Price',
           'Model_std_Price']
for col in collist:
    train[col].fillna(0, inplace=True)
    test[col].fillna(0, inplace=True)

In [546]:
for col in [["Manufacturer",'Location','Seats'],["Model",'Location','Seats']]:
    mdf = temp_zero.groupby(col)['New_Price'].mean().reset_index()
    if "Model" in col:
        mdf.columns = col + ["Model_Loc_Seats_mean_Price"]
    else:
        mdf.columns = col + ["Manu_Loc_Seats_mean_Price"]
    
    train = pd.merge(train, mdf, on=col, how="left")
    test = pd.merge(test, mdf, on=col, how="left")
    
    sdf = temp_zero.groupby(col)['New_Price'].std().reset_index()
    if "Model" in col:
        sdf.columns = col + ["Model_Loc_Seats_std_Price"]
    else:
        sdf.columns = col + ["Manu_Loc_Seats_std_Price"]
    
    train = pd.merge(train, sdf, on=col, how="left")
    test = pd.merge(test, sdf, on=col, how="left")
    
    cdf = temp_zero.groupby(col)['New_Price'].count().reset_index()
    if "Model" in col:
        cdf.columns = col + ["Model_Loc_Seats_count"]
    else:
        cdf.columns = col + ["Manu_Loc_Seats_count"]
    
    train = pd.merge(train, cdf, on=col, how="left")
    test = pd.merge(test, cdf, on=col, how="left")

In [547]:
for col in ["Model_Loc_Seats_mean_Price","Manu_Loc_Seats_mean_Price","Model_Loc_Seats_std_Price","Manu_Loc_Seats_std_Price",
           "Model_Loc_Seats_count","Manu_Loc_Seats_count"]:
    train[col].fillna(0, inplace=True)
    test[col].fillna(0, inplace=True)

In [548]:
for col in ['Owner_Type','Transmission','Fuel_Type']:
    temp =  train[['Model','Owner_Type','Transmission','Fuel_Type','Model_mean_Price']]
#    mdf = pd.pivot_table(temp_zero, index="Model", columns=col, values="New_Price", aggfunc="mean", fill_value=0).reset_index()
    mdf = pd.pivot_table(temp, index="Model", columns=col, values="Model_mean_Price", aggfunc="mean", fill_value=0).reset_index()
    train = pd.merge(train, mdf, on=["Model"], how="left")
    test = pd.merge(test, mdf, on=["Model"], how="left")
    
    mdf = pd.pivot_table(temp, index="Model", columns=col, values="Model_mean_Price", aggfunc="std", fill_value=0).reset_index()
    train = pd.merge(train, mdf, on=["Model"], how="left")
    test = pd.merge(test, mdf, on=["Model"], how="left")

In [549]:
col_list = ['Location','Transmission','Owner_Type','Fuel_Type','Model','Manufacturer','Manu_Model']
for col in col_list:
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(train[col].values.astype('str')) + list(test[col].values.astype('str')))
        train[col] = lbl.fit_transform(list(train[col].values.astype('str')))
        test[col] = lbl.fit_transform(list(test[col].values.astype('str')))

In [550]:
train.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price,Manufacturer,Model,Manu_Model,old_by,Manufacturer_Seats_mean_Price,Manufacturer_Seats_std_Price,Model_Seats_mean_Price,Model_Seats_std_Price,Manu_Model_Seats_mean_Price,Manu_Model_Seats_std_Price,Model_Year_mean_Price,Manufacturer_mean_Price,Manufacturer_std_Price,Model_mean_Price,Model_std_Price,Manu_Loc_Seats_mean_Price,Manu_Loc_Seats_std_Price,Manu_Loc_Seats_count,Model_Loc_Seats_mean_Price,Model_Loc_Seats_std_Price,Model_Loc_Seats_count,First_x,Fourth & Above_x,Second_x,Third_x,First_y,Fourth & Above_y,Second_y,Third_y,Automatic_x,Manual_x,Automatic_y,Manual_y,CNG_x,Diesel_x,Electric_x,LPG_x,Petrol_x,CNG_y,Diesel_y,Electric_y,LPG_y,Petrol_y
0,maruti wagon r lxi cng,9,2010,72000,0,1,0,26.600000,998,58.16,5.0,0.00,1.75,17,188,123,9,7.784929,2.654674,5.469556,0.537676,5.469556,0.537676,5.188,7.845907,2.746784,5.469556,0.537676,7.523235,2.262745,34.0,5.740833,0.563616,12.0,5.469556,0.0,5.469556,5.469556,0,0,0,0,5.469556,5.469556,0,0,5.469556,0.000000,0,5.469556,5.469556,0,0,0,0,0
1,hyundai creta 1.6 crdi sx option,10,2015,41000,1,1,0,23.641827,1582,126.20,5.0,0.00,12.50,10,50,62,4,11.836154,5.269123,16.415000,2.074013,16.415000,2.074013,18.320,11.836154,5.269123,16.415000,2.074013,13.111667,6.051933,6.0,18.320000,0.000000,1.0,16.415000,0.0,16.415000,0.000000,0,0,0,0,16.415000,16.415000,0,0,0.000000,16.415000,0,0.000000,16.415000,0,0,0,0,0
2,honda jazz v,2,2011,46000,4,1,0,24.594595,1199,88.70,5.0,8.61,4.50,9,99,58,8,11.112143,4.162726,9.750714,0.995200,9.750714,0.995200,8.610,11.669759,4.055201,9.750714,0.995200,13.636000,11.234304,5.0,9.506667,1.155004,3.0,9.750714,0.0,9.750714,0.000000,0,0,0,0,9.750714,9.750714,0,0,0.000000,9.750714,0,0.000000,9.750714,0,0,0,0,0
3,maruti ertiga vdi,2,2012,87000,1,1,0,24.963942,1248,88.76,7.0,0.00,6.00,17,65,111,7,8.647500,3.388862,11.384286,0.967906,11.384286,0.967906,0.000,7.845907,2.746784,11.384286,0.967906,7.930000,4.327494,2.0,10.990000,0.000000,1.0,11.384286,0.0,11.384286,0.000000,0,0,0,0,11.384286,11.384286,0,0,11.384286,11.384286,0,0.000000,11.384286,0,0,0,0,0
4,audi a4 new 2.0 tdi multitronic,3,2013,40670,1,0,2,18.269231,1968,140.80,5.0,0.00,17.74,1,10,2,6,53.643913,8.432411,52.976250,2.960603,52.976250,2.960603,53.140,65.141724,24.946476,52.976250,2.960603,51.335000,8.338979,8.0,53.140000,0.000000,3.0,52.976250,0.0,52.976250,0.000000,0,0,0,0,52.976250,0.000000,0,0,0.000000,52.976250,0,0.000000,52.976250,0,0,0,0,0


In [551]:
encoded_output = train['Price']
train.drop(['Name','Year','Price'], axis=1, inplace=True)
test.drop(['Name','Year'], axis=1, inplace=True)

In [552]:
train.fillna(0, inplace=True)
test.fillna(0, inplace=True)

In [553]:
ss = preprocessing.StandardScaler()
encoded_train = pd.DataFrame(ss.fit_transform(train),columns=train.columns)
encoded_test = pd.DataFrame(ss.fit_transform(test),columns=test.columns)

In [554]:
X_train, X_test, y_train, y_test = train_test_split(encoded_train, encoded_output, test_size=0.30, random_state=42)

In [555]:
class ReduceVIF(BaseEstimator, TransformerMixin):
    def __init__(self, thresh=15.0):
        # From looking at documentation, values between 5 and 10 are "okay".
        # Above 10 is too high and so should be removed.
        self.thresh = thresh
        
    def fit(self, X, y=None):
        print('ReduceVIF fit')
        return self

    def transform(self, X, y=None):
        print('ReduceVIF transform')
        columns = X.columns.tolist()
        return ReduceVIF.calculate_vif(X, self.thresh)

    @staticmethod
    def calculate_vif(X, thresh=15.0):
        # Taken from https://stats.stackexchange.com/a/253620/53565 and modified
        dropped=True
        while dropped:
            variables = X.columns
            dropped = False
            vif = [variance_inflation_factor(X[variables].values, X.columns.get_loc(var)) for var in X.columns]
            
            max_vif = max(vif)
            if max_vif > thresh:
                maxloc = vif.index(max_vif)
                print('Dropping {} with vif={}'.format(X.columns[maxloc],max_vif))
                X = X.drop([X.columns.tolist()[maxloc]], axis=1)
                dropped=True
        return X

In [556]:
transformer = ReduceVIF()
X = transformer.fit_transform(X_train, y_train)

ReduceVIF fit
ReduceVIF transform


C:\Anaconda\envs\tfdeeplearning\lib\site-packages\statsmodels\stats\outliers_influence.py:181: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
C:\Anaconda\envs\tfdeeplearning\lib\site-packages\statsmodels\regression\linear_model.py:1545: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.uncentered_tss


Dropping Model_Seats_mean_Price with vif=inf
Dropping Model_mean_Price with vif=inf
Dropping Manu_Model_Seats_std_Price with vif=56714.44198033447
Dropping First_x with vif=277.61487903040177
Dropping Manu_Model with vif=168.07462116209595
Dropping Manu_Model_Seats_mean_Price with vif=76.5783729608412
Dropping Automatic_x with vif=32.746804231605545
Dropping Diesel_x with vif=17.39878103037592


In [557]:
#X_train = X_train[X.columns]
#X_test = X_test[X.columns]
#encoded_test = encoded_test[X.columns]

In [558]:
parameters = {
    'learning_rate':[0.01,0.4],
    'max_depth':[5,6,7],
    'n_estimators':[50,75,100],
    'min_samples_split':[100,150,200],
    'min_samples_leaf':[20,26,30]
}

GSV = GridSearchCV(GradientBoostingRegressor(),parameters,cv=5)
GSV.fit(X_train, np.log10(y_train))
clf = GSV.best_estimator_
clf.fit(X_train, np.log10(y_train))
print (1-mean_squared_log_error(y_test, 10**clf.predict(X_test)))
print (1-mean_squared_log_error(y_train, 10**clf.predict(X_train)))
prediction = 10**clf.predict(encoded_test)

0.971879553297
0.991700833011


In [559]:
model_output = pd.DataFrame(10**clf.predict(encoded_test),columns=['Price'])
model_output.to_excel('GrdRgr_Grid.xlsx', index=False)

In [560]:
svr=SVR(kernel='linear',degree=1)
svr.fit(X_train, np.log10(y_train))
print (1-mean_squared_log_error(y_test, 10**svr.predict(X_test)))
print (1-mean_squared_log_error(y_train, 10**svr.predict(X_train)))
prediction = 10**svr.predict(encoded_test)

0.924758319312
0.951053061968


In [561]:
model_output = pd.DataFrame(10**svr.predict(encoded_test),columns=['Price'])
model_output.to_excel('SVRRgr3_v1.xlsx', index=False)